## 라이브러리 및 함수 선언

In [1]:
import os
from datetime import datetime

import h5py
import numpy as np
import pandas as pd

In [2]:
def get_file_list(file_path, extension):
    file_list = []
    for path, dirs, files in os.walk(file_path):
        path = path.replace("\\", "/")
        for file in files:
            if os.path.splitext(file)[1].lower() == extension:
                file_list.append("{}/{}".format(path, file))
    
    return file_list

In [3]:
def dummies_columns(columns_name, dataframe):
    dummies_one_hot = pd.get_dummies(dataframe[columns_name], prefix=columns_name)
    dataframe = pd.concat([pd.DataFrame(dataframe), dummies_one_hot], axis=1)
    
    return list(dummies_one_hot), dataframe

## 앨범 메타데이터 가져오기

In [4]:
music_info_df = pd.read_csv("../data/meta/bugs_music_info.csv").rename(columns={"장르": "앨범 장르"})
music_info_df = music_info_df[music_info_df["발매일"].notnull()].reset_index(drop=True)
music_info_df["발매일_datetime"] = music_info_df["발매일"].apply(
    lambda x: datetime.strptime(x, "%Y.%m") if len(x) < 10 else datetime.strptime(x, "%Y.%m.%d"))
music_info_df

,sing_id,subject,singer,앨범 종류,발매일,앨범 장르,스타일,기획사,유통사,발매일_datetime
0,89688529,십이월 이십오일의 고백,정승환,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05
1,89613039,Blueming,아이유 (IU),EP(미니),2019.11.18,댄스/팝,발라드,카카오엔터테인먼트,카카오엔터테인먼트,2019-11-18
2,89682091,아마두 (Feat. 우원재 & 김효은 & 넉살 & Huckleberry P),염따 & 딥플로우 & 팔로알토 (Paloalto) & The Quiett & 사이먼...,싱글,2019.12.03,랩/힙합,랩,"AOMG, 딩고뮤직, Hi-Lite Records, ILLIONAIRE RECORD...",Dreamus,2019-12-03
3,89678583,나의 오랜 연인에게,다비치,싱글,2019.12.03,발라드,발라드,WAKEONE,지니뮤직,2019-12-03
4,89602794,Into the Unknown,Idina Menzel & AURORA,싱글,2019.12.13,팝,팝,Sing2Music Productions Pty Ltd,FUGA,2019-12-13
...,...,...,...,...,...,...,...,...,...,...
6594,94618083,Easy On Me,Adele,싱글,2021.10.14,팝,소울 팝,Columbia,Sony Music,2021-10-14
6595,94609018,우리의 이야기,멜로망스 (MeloMance),싱글,2021.10.15,인디,인디 팝,Stone Music Entertainment,지니뮤직,2021-10-15
6596,94611799,To You (Prod. by 정키),HYNN(박혜원) & 정키,EP(미니),2021.10.15,발라드,발라드,뉴오더 엔터테인먼트,카카오엔터테인먼트,2021-10-15
6597,94611800,남자가 되면,HYNN (박혜원),EP(미니),2021.10.15,발라드,발라드,뉴오더 엔터테인먼트,카카오엔터테인먼트,2021-10-15


## 앨범 차트 데이터 가져오기

In [5]:
file_path = "../data/guysome_genie_realchart"
extension = ".csv"
chart_file_list = get_file_list(file_path, extension)
chart_file_list

['../data/guysome_genie_realchart/20211015.csv',
 '../data/guysome_genie_realchart/20211014.csv',
 '../data/guysome_genie_realchart/20211013.csv',
 '../data/guysome_genie_realchart/20211012.csv',
 '../data/guysome_genie_realchart/20211011.csv',
 '../data/guysome_genie_realchart/20211010.csv',
 '../data/guysome_genie_realchart/20211009.csv',
 '../data/guysome_genie_realchart/20211008.csv',
 '../data/guysome_genie_realchart/20211007.csv',
 '../data/guysome_genie_realchart/20211006.csv',
 '../data/guysome_genie_realchart/20211005.csv',
 '../data/guysome_genie_realchart/20211004.csv',
 '../data/guysome_genie_realchart/20211003.csv',
 '../data/guysome_genie_realchart/20211002.csv',
 '../data/guysome_genie_realchart/20211001.csv',
 '../data/guysome_genie_realchart/20210930.csv',
 '../data/guysome_genie_realchart/20210929.csv',
 '../data/guysome_genie_realchart/20210928.csv',
 '../data/guysome_genie_realchart/20210927.csv',
 '../data/guysome_genie_realchart/20210926.csv',
 '../data/guysome_ge

In [6]:
chart_all_df = pd.DataFrame()

for chart_file in chart_file_list:
    chart_df = pd.read_csv(chart_file, encoding="cp949")
    chart_all_df = pd.concat([chart_all_df, chart_df], axis=0)
    
chart_all_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,listen_total
0,2021/10/15/0,1,-,94500050,Savage,aespa,1,11276.0,665.0,4128985,655681
1,2021/10/15/0,2,2,94535712,사랑은 늘 도망가,임영웅,1,27077.0,891.0,2040357,132656
2,2021/10/15/0,3,1,93717294,Stay,The Kid LAROI & Justin Bieber,1,6365.0,136.0,23801656,1479557
3,2021/10/15/0,4,1,94395717,My Universe,Coldplay & 방탄소년단,1,6377.0,298.0,6961273,900753
4,2021/10/15/0,5,6,90077755,이제 나만 믿어요,임영웅,1,14444.0,35.0,167890047,2032066
...,...,...,...,...,...,...,...,...,...,...,...
3245,2019/12/08/23,246,4,89466146,LOVE ME,뉴이스트,1,2895.0,31.0,8389233,323098
3246,2019/12/08/23,247,16,88562939,이 노래가 클럽에서 나온다면,우디 (Woody),1,1024.0,21.0,43388473,1890978
3247,2019/12/08/23,248,NEW,88740321,Kill This Love,BLACKPINK,2,2144.0,52.0,33736583,1731648
3248,2019/12/08/23,249,33,89602839,기억의 강 (겨울왕국 2),조영경 (Cho Young Kyoung),1,1909.0,199.0,435140,88266


## 데이터 병합 (앨범 메타 + 차트)

In [7]:
chart_info_df = pd.merge(chart_all_df, music_info_df, on=["sing_id", "subject", "singer"])
chart_info_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,listen_total,앨범 종류,발매일,앨범 장르,스타일,기획사,유통사,발매일_datetime
0,2021/10/15/0,1,-,94500050,Savage,aespa,1,11276.0,665.0,4128985,655681,EP(미니),2021.10.05,댄스/팝,댄스 팝,SM ENTERTAINMENT,Dreamus,2021-10-05
1,2021/10/15/1,2,1,94500050,Savage,aespa,1,7487.0,474.0,4136472,656155,EP(미니),2021.10.05,댄스/팝,댄스 팝,SM ENTERTAINMENT,Dreamus,2021-10-05
2,2021/10/15/2,2,1,94500050,Savage,aespa,1,5081.0,257.0,4141553,656412,EP(미니),2021.10.05,댄스/팝,댄스 팝,SM ENTERTAINMENT,Dreamus,2021-10-05
3,2021/10/15/3,2,1,94500050,Savage,aespa,1,3378.0,157.0,4144931,656569,EP(미니),2021.10.05,댄스/팝,댄스 팝,SM ENTERTAINMENT,Dreamus,2021-10-05
4,2021/10/15/4,2,1,94500050,Savage,aespa,1,2476.0,118.0,4147407,656687,EP(미니),2021.10.05,댄스/팝,댄스 팝,SM ENTERTAINMENT,Dreamus,2021-10-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3922044,2019/12/08/14,179,29,89694352,그게 우리의 마지막인 줄 알았다면 (With 성영주),새봄 (Saevom),1,2250.0,1572.0,3913,2950,싱글,2019.12.08,포크/어쿠스틱,발라드,어느새봄,포크라노스,2019-12-08
3922045,2019/12/08/15,182,3,89694352,그게 우리의 마지막인 줄 알았다면 (With 성영주),새봄 (Saevom),1,2299.0,1525.0,6212,4475,싱글,2019.12.08,포크/어쿠스틱,발라드,어느새봄,포크라노스,2019-12-08
3922046,2019/12/08/16,217,35,89694352,그게 우리의 마지막인 줄 알았다면 (With 성영주),새봄 (Saevom),1,2116.0,1325.0,8328,5800,싱글,2019.12.08,포크/어쿠스틱,발라드,어느새봄,포크라노스,2019-12-08
3922047,2019/12/08/17,218,1,89694352,그게 우리의 마지막인 줄 알았다면 (With 성영주),새봄 (Saevom),1,2199.0,1320.0,10527,7120,싱글,2019.12.08,포크/어쿠스틱,발라드,어느새봄,포크라노스,2019-12-08


## 설명변수 생성

In [8]:
# date_time (str -> datetime 형식으로)
chart_info_df["date_time"] = chart_info_df["date"].apply(lambda x: datetime.strptime(x, "%Y/%m/%d/%H"))
chart_info_df = chart_info_df[chart_info_df["발매일_datetime"] <= chart_info_df["date_time"]].reset_index(drop=True)

chart_info_df = chart_info_df.sort_values(["date_time", "rank"]).reset_index(drop=True)
chart_info_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,listen_total,앨범 종류,발매일,앨범 장르,스타일,기획사,유통사,발매일_datetime,date_time
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,717398,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2019-12-08 11:00:00
1,2019/12/08/11,2,-,89613039,Blueming,아이유 (IU),0,NaN,NaN,16624334,1528860,EP(미니),2019.11.18,댄스/팝,발라드,카카오엔터테인먼트,카카오엔터테인먼트,2019-11-18,2019-12-08 11:00:00
2,2019/12/08/11,3,-,89682091,아마두 (Feat. 우원재 & 김효은 & 넉살 & Huckleberry P),염따 & 딥플로우 & 팔로알토 (Paloalto) & The Quiett & 사이먼...,0,NaN,NaN,2971878,710124,싱글,2019.12.03,랩/힙합,랩,"AOMG, 딩고뮤직, Hi-Lite Records, ILLIONAIRE RECORD...",Dreamus,2019-12-03,2019-12-08 11:00:00
3,2019/12/08/11,4,-,89678583,나의 오랜 연인에게,다비치,0,NaN,NaN,3397457,899346,싱글,2019.12.03,발라드,발라드,WAKEONE,지니뮤직,2019-12-03,2019-12-08 11:00:00
4,2019/12/08/11,6,1,89558702,늦은 밤 너의 집 앞 골목길에서,노을,0,NaN,NaN,20911975,1543733,싱글,2019.11.07,발라드,발라드,씨제스엔터테인먼트,카카오엔터테인먼트,2019-11-07,2019-12-08 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3863464,2021/10/15/23,246,NEW,85912311,회상,장범준,11,3835.0,370.0,13424706,1172123,OST,2016.01.30,발라드,발라드,Stone Music Entertainment,지니뮤직,2016-01-30,2021-10-15 23:00:00
3863465,2021/10/15/23,247,NEW,94611798,내 사랑 (Duet. 카더가든),HYNN(박혜원) & 카더가든,0,NaN,NaN,5261,4239,EP(미니),2021.10.15,발라드,발라드,뉴오더 엔터테인먼트,카카오엔터테인먼트,2021-10-15,2021-10-15 23:00:00
3863466,2021/10/15/23,248,5,94515928,Here Always (승민 of Stray Kids),Stray Kids (스트레이 키즈),1,1232.0,181.0,272456,61631,OST,2021.10.10,발라드,발라드,Stone Music Entertainment,지니뮤직,2021-10-10,2021-10-15 23:00:00
3863467,2021/10/15/23,249,11,89238855,그대라는 시,태연 (TAEYEON),1,833.0,24.0,64934779,2653064,OST,2019.07.21,발라드,발라드,냠냠엔터테인먼트,Dreamus,2019-07-21,2021-10-15 23:00:00


In [9]:
# 날짜 설명변수 생성 (year, month, hour)
chart_info_df["date_time_year"] = chart_info_df["date_time"].dt.year
chart_info_df["date_time_month"] = chart_info_df["date_time"].dt.month
chart_info_df["date_time_hour"] = chart_info_df["date_time"].dt.hour
chart_info_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,...,발매일,앨범 장르,스타일,기획사,유통사,발매일_datetime,date_time,date_time_year,date_time_month,date_time_hour
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,...,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2019-12-08 11:00:00,2019,12,11
1,2019/12/08/11,2,-,89613039,Blueming,아이유 (IU),0,NaN,NaN,16624334,...,2019.11.18,댄스/팝,발라드,카카오엔터테인먼트,카카오엔터테인먼트,2019-11-18,2019-12-08 11:00:00,2019,12,11
2,2019/12/08/11,3,-,89682091,아마두 (Feat. 우원재 & 김효은 & 넉살 & Huckleberry P),염따 & 딥플로우 & 팔로알토 (Paloalto) & The Quiett & 사이먼...,0,NaN,NaN,2971878,...,2019.12.03,랩/힙합,랩,"AOMG, 딩고뮤직, Hi-Lite Records, ILLIONAIRE RECORD...",Dreamus,2019-12-03,2019-12-08 11:00:00,2019,12,11
3,2019/12/08/11,4,-,89678583,나의 오랜 연인에게,다비치,0,NaN,NaN,3397457,...,2019.12.03,발라드,발라드,WAKEONE,지니뮤직,2019-12-03,2019-12-08 11:00:00,2019,12,11
4,2019/12/08/11,6,1,89558702,늦은 밤 너의 집 앞 골목길에서,노을,0,NaN,NaN,20911975,...,2019.11.07,발라드,발라드,씨제스엔터테인먼트,카카오엔터테인먼트,2019-11-07,2019-12-08 11:00:00,2019,12,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3863464,2021/10/15/23,246,NEW,85912311,회상,장범준,11,3835.0,370.0,13424706,...,2016.01.30,발라드,발라드,Stone Music Entertainment,지니뮤직,2016-01-30,2021-10-15 23:00:00,2021,10,23
3863465,2021/10/15/23,247,NEW,94611798,내 사랑 (Duet. 카더가든),HYNN(박혜원) & 카더가든,0,NaN,NaN,5261,...,2021.10.15,발라드,발라드,뉴오더 엔터테인먼트,카카오엔터테인먼트,2021-10-15,2021-10-15 23:00:00,2021,10,23
3863466,2021/10/15/23,248,5,94515928,Here Always (승민 of Stray Kids),Stray Kids (스트레이 키즈),1,1232.0,181.0,272456,...,2021.10.10,발라드,발라드,Stone Music Entertainment,지니뮤직,2021-10-10,2021-10-15 23:00:00,2021,10,23
3863467,2021/10/15/23,249,11,89238855,그대라는 시,태연 (TAEYEON),1,833.0,24.0,64934779,...,2019.07.21,발라드,발라드,냠냠엔터테인먼트,Dreamus,2019-07-21,2021-10-15 23:00:00,2021,10,23


In [10]:
# 시간 설명변수 생성 (발매일-현재)
chart_info_df["duration"] = chart_info_df["date_time"] - chart_info_df["발매일_datetime"]
chart_info_df["duration_days"] = chart_info_df["duration"].apply(lambda x: x.days)
chart_info_df["duration_seconds"] = chart_info_df["duration"].apply(lambda x: x.seconds)
chart_info_df["duration_hours"] = chart_info_df["duration_days"] * 24 + chart_info_df["duration_seconds"] // 3600
chart_info_df = chart_info_df[chart_info_df["duration_hours"] <= 8760].reset_index(drop=True)
chart_info_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,...,유통사,발매일_datetime,date_time,date_time_year,date_time_month,date_time_hour,duration,duration_days,duration_seconds,duration_hours
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,...,카카오엔터테인먼트,2019-12-05,2019-12-08 11:00:00,2019,12,11,3 days 11:00:00,3,39600,83
1,2019/12/08/11,2,-,89613039,Blueming,아이유 (IU),0,NaN,NaN,16624334,...,카카오엔터테인먼트,2019-11-18,2019-12-08 11:00:00,2019,12,11,20 days 11:00:00,20,39600,491
2,2019/12/08/11,3,-,89682091,아마두 (Feat. 우원재 & 김효은 & 넉살 & Huckleberry P),염따 & 딥플로우 & 팔로알토 (Paloalto) & The Quiett & 사이먼...,0,NaN,NaN,2971878,...,Dreamus,2019-12-03,2019-12-08 11:00:00,2019,12,11,5 days 11:00:00,5,39600,131
3,2019/12/08/11,4,-,89678583,나의 오랜 연인에게,다비치,0,NaN,NaN,3397457,...,지니뮤직,2019-12-03,2019-12-08 11:00:00,2019,12,11,5 days 11:00:00,5,39600,131
4,2019/12/08/11,6,1,89558702,늦은 밤 너의 집 앞 골목길에서,노을,0,NaN,NaN,20911975,...,카카오엔터테인먼트,2019-11-07,2019-12-08 11:00:00,2019,12,11,31 days 11:00:00,31,39600,755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522338,2021/10/15/23,238,11,92791595,BAD LOVE,BE'O (비오),1,1038.0,135.0,187317,...,"지니뮤직, Stone Music Entertainment",2021-03-30,2021-10-15 23:00:00,2021,10,23,199 days 23:00:00,199,82800,4799
2522339,2021/10/15/23,243,11,93154014,마.피.아. In the morning,ITZY (있지),1,800.0,22.0,13265044,...,Dreamus,2021-04-30,2021-10-15 23:00:00,2021,10,23,168 days 23:00:00,168,82800,4055
2522340,2021/10/15/23,247,NEW,94611798,내 사랑 (Duet. 카더가든),HYNN(박혜원) & 카더가든,0,NaN,NaN,5261,...,카카오엔터테인먼트,2021-10-15,2021-10-15 23:00:00,2021,10,23,0 days 23:00:00,0,82800,23
2522341,2021/10/15/23,248,5,94515928,Here Always (승민 of Stray Kids),Stray Kids (스트레이 키즈),1,1232.0,181.0,272456,...,지니뮤직,2021-10-10,2021-10-15 23:00:00,2021,10,23,5 days 23:00:00,5,82800,143


In [11]:
# 차트 인(rank_out_term) 유지 시간이 720시간 이상인 곡만 추림
sing_id_dict = {}

sing_id_list = chart_info_df["sing_id"].unique()

for s_idx, sing_id in enumerate(sing_id_list):
    compare_idx = -10
    compare_idx_list = []
    compare_tmp_list = []

    chart_one_df = chart_info_df[chart_info_df["sing_id"] == sing_id].reset_index(drop=True)
    df_index_list = chart_one_df[chart_one_df["rank_out_term"] == 1].index

    for e_idx, df_idx in enumerate(df_index_list):
        if compare_idx + 1 == df_idx:
            compare_tmp_list.append(df_idx)
        elif compare_idx != df_idx + 1 or e_idx == len(df_index_list) - 1:
            if len(compare_tmp_list) > 720:
                compare_idx_list = compare_idx_list + [compare_tmp_list]
            compare_tmp_list = []
        compare_idx = df_idx

    if compare_idx_list:
        sing_id_dict[sing_id] = compare_idx_list
        print("=" * 50)
        print(" sing_id: {}\n 제목: {}\n 가수: {}\n index cnt: {}".format(
            sing_id, chart_one_df.iloc[0]["subject"], chart_one_df.iloc[0]["singer"],
            [len(_) for _ in compare_idx_list]))
        
chart_info_df = chart_info_df[chart_info_df["sing_id"].isin(sing_id_dict.keys())].reset_index(drop=True)
chart_info_df

 sing_id: 89688529
 제목: 십이월 이십오일의 고백
 가수: 정승환
 index cnt: [728, 1119]
 sing_id: 89613039
 제목: Blueming
 가수: 아이유 (IU)
 index cnt: [728, 1119, 2924, 2107]
 sing_id: 89682091
 제목: 아마두 (Feat. 우원재 & 김효은 & 넉살 & Huckleberry P)
 가수: 염따 & 딥플로우 & 팔로알토 (Paloalto) & The Quiett & 사이먼 도미닉
 index cnt: [728, 1119, 1919]
 sing_id: 89678583
 제목: 나의 오랜 연인에게
 가수: 다비치
 index cnt: [728, 1119, 2924]
 sing_id: 89558702
 제목: 늦은 밤 너의 집 앞 골목길에서
 가수: 노을
 index cnt: [728, 1119, 2924, 2107]
 sing_id: 89598605
 제목: HIP
 가수: 마마무(Mamamoo)
 index cnt: [728, 1119, 2924, 1513]
 sing_id: 89671315
 제목: 헤어진 우리가 지켜야 할 것들
 가수: 김나영 & 양다일
 index cnt: [728, 1119, 1584]
 sing_id: 89530447
 제목: Love poem
 가수: 아이유 (IU)
 index cnt: [728, 1119, 2924, 2107]
 sing_id: 89382909
 제목: 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지
 가수: AKMU (악뮤)
 index cnt: [728, 1119, 2924]
 sing_id: 89320910
 제목: 흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야
 가수: 장범준
 index cnt: [728, 1119, 2924]
 sing_id: 89602798
 제목: Show Yourself
 가수: Idina Menzel & Evan Rachel Wood
 index cnt: [728, 764]
 

 sing_id: 88686378
 제목: 주저하는 연인들을 위해
 가수: 잔나비
 index cnt: [728]
 sing_id: 89290483
 제목: 내 목소리 들리니
 가수: 벤
 index cnt: [728, 1119]
 sing_id: 89374289
 제목: Memories
 가수: Maroon 5
 index cnt: [728, 741, 2924]
 sing_id: 89378490
 제목: 나만 아픈 일
 가수: 이해리 (다비치)
 index cnt: [728, 864]
 sing_id: 89369155
 제목: 가끔 이러다
 가수: 펀치 (Punch)
 index cnt: [728]
 sing_id: 89391731
 제목: 천둥벌거숭이 (Feat. Jvcki Wai & 염따)
 가수: 지코 (ZICO)
 index cnt: [728]
 sing_id: 88824340
 제목: FANCY
 가수: TWICE (트와이스)
 index cnt: [728, 1119]
 sing_id: 88715422
 제목: 안녕이라는 말
 가수: 송이한
 index cnt: [728]
 sing_id: 89258787
 제목: ICY
 가수: ITZY (있지)
 index cnt: [728]
 sing_id: 89408689
 제목: 이별행동
 가수: 이우
 index cnt: [728, 1119]
 sing_id: 89623205
 제목: 우리 왜 헤어져야 해
 가수: 신예영
 index cnt: [728, 740, 2924, 2107]
 sing_id: 89226517
 제목: 내 안부
 가수: 이우
 index cnt: [728, 842]
 sing_id: 88773174
 제목: 소우주 (Mikrokosmos)
 가수: 방탄소년단
 index cnt: [728]
 sing_id: 89671630
 제목: 빌었어
 가수: 창모 (CHANGMO)
 index cnt: [728, 1119, 2924]
 sing_id: 88728543
 제목: 사월이 지나면 우

 sing_id: 89831306
 제목: 일편단심 민들레야
 가수: 임영웅
 index cnt: [1290]
 sing_id: 89290354
 제목: Dance Monkey
 가수: Tones And I
 index cnt: [2924]
 sing_id: 89830405
 제목: 돌덩이
 가수: 하현우 (국카스텐)
 index cnt: [927, 2924, 1513]
 sing_id: 89831932
 제목: 둘만의 세상으로 가
 가수: Crush
 index cnt: [1085, 1583]
 sing_id: 89831917
 제목: 잘 지내고 있는지 궁금해
 가수: V.O.S
 index cnt: [1085, 2924, 1348]
 sing_id: 89859975
 제목: 마음을 드려요
 가수: 아이유 (IU)
 index cnt: [941, 2924, 2107]
 sing_id: 89858874
 제목: 그때 그 아인
 가수: 김필
 index cnt: [941, 2924, 2107]
 sing_id: 89863137
 제목: FIESTA
 가수: IZ*ONE (아이즈원)
 index cnt: [893, 2924, 1175]
 sing_id: 89786517
 제목: Black Swan
 가수: 방탄소년단
 index cnt: [817, 2924, 1565]
 sing_id: 89894708
 제목: 어느 60대 노부부이야기
 가수: 임영웅
 index cnt: [2731, 2107]
 sing_id: 89883239
 제목: Filter
 가수: 방탄소년단
 index cnt: [797, 2574, 1489]
 sing_id: 89883240
 제목: 시차
 가수: 방탄소년단
 index cnt: [797, 2219, 881]
 sing_id: 89883243
 제목: 욱 (UGH!)
 가수: 방탄소년단
 index cnt: [797, 2051]
 sing_id: 89883241
 제목: Louder than bombs
 가수: 방탄소년단
 index

 sing_id: 90161044
 제목: 내 눈물 모아
 가수: 휘인 (Whee In)
 index cnt: [2053, 2107, 1118]
 sing_id: 90173969
 제목: Happy
 가수: 태연 (TAEYEON)
 index cnt: [1975, 815]
 sing_id: 90173917
 제목: 은하수
 가수: 길구봉구
 index cnt: [874]
 sing_id: 90146199
 제목: 서면역에서
 가수: 순순희
 index cnt: [972, 3889]
 sing_id: 90178428
 제목: 에잇 (Prod. & Feat. SUGA of BTS)
 가수: 아이유 (IU)
 index cnt: [1927, 2107]
 sing_id: 88323996
 제목: Comethru
 가수: Jeremy Zucker
 index cnt: [1899, 2107, 1485, 1005]
 sing_id: 90194896
 제목: Zombie
 가수: DAY6 (데이식스)
 index cnt: [1807]
 sing_id: 90198070
 제목: I'm in Trouble
 가수: 뉴이스트
 index cnt: [784]
 sing_id: 90182881
 제목: 나비와 고양이 (Feat. 백현 (BAEKHYUN))
 가수: 볼빨간사춘기
 index cnt: [1779, 1513]
 sing_id: 90208247
 제목: 품
 가수: 볼빨간사춘기
 index cnt: [1759, 1007]
 sing_id: 90210693
 제목: 밤이 깊었네 (Drama Ver.)
 가수: 미도와 파라솔
 index cnt: [1384]
 sing_id: 90235529
 제목: 자나깨나 (Feat. 조이 of Red Velvet)
 가수: Crush
 index cnt: [1591]
 sing_id: 90250302
 제목: 사랑하게 될 줄 알았어
 가수: 전미도
 index cnt: [1549, 2107, 3905]
 sing_id: 90253075
 

 sing_id: 90641114
 제목: 나가
 가수: 먼데이 키즈 (Monday Kiz) & 이예준
 index cnt: [784]
 sing_id: 90661587
 제목: When We Disco (Duet With 선미)
 가수: 박진영
 index cnt: [1686, 3785]
 sing_id: 90677250
 제목: 오래된 노래
 가수: 임영웅
 index cnt: [1643, 2580]
 sing_id: 90676965
 제목: Not Shy
 가수: ITZY (있지)
 index cnt: [1566, 839]
 sing_id: 90705547
 제목: Dynamite
 가수: 방탄소년단
 index cnt: [1475]
 sing_id: 90709386
 제목: 안녕 못 해
 가수: 김재환
 index cnt: [1390]
 sing_id: 90709438
 제목: 내 마음이 움찔했던 순간 (취향저격 그녀 X 규현)
 가수: 규현 (KYUHYUN)
 index cnt: [1422, 5952]
 sing_id: 90743704
 제목: 듣고 싶던 말
 가수: 허각
 index cnt: [1325]
 sing_id: 90751091
 제목: Ice Cream (With Selena Gomez)
 가수: BLACKPINK
 index cnt: [1307]
 sing_id: 90751198
 제목: 그 중에 그대를 만나
 가수: 임영웅
 index cnt: [1251]
 sing_id: 90751194
 제목: 그런 사람 또 없습니다
 가수: 임영웅
 index cnt: [832]
 sing_id: 90788463
 제목: 만개 (Prod. 신지후)
 가수: 김호중
 index cnt: [1110, 7026]
 sing_id: 90788465
 제목: 우산이 없어요
 가수: 김호중
 index cnt: [1110, 805, 1075, 826, 1071]
 sing_id: 90788461
 제목: 홀로아리랑 (Prod. by 박경훈)
 가수: 김호중

 sing_id: 91466463
 제목: 혼술하고 싶은 밤
 가수: 벤
 index cnt: [4282]
 sing_id: 91552332
 제목: 한 번에 알아본 사랑 (바른연애 길잡이 X 양요섭)
 가수: 양요섭
 index cnt: [1154]
 sing_id: 91551008
 제목: 그날에 나는 맘이 편했을까
 가수: 이예준
 index cnt: [5339, 1071]
 sing_id: 91563412
 제목: 내일이 오면 (Feat. 기리보이 & BIG Naughty (서동현))
 가수: 릴보이 (lIlBOI)
 index cnt: [3352]
 sing_id: 91563409
 제목: Achoo (Feat. pH-1 & HAON) (Prod. by GroovyRoom)
 가수: 미란이 (Mirani)
 index cnt: [2392]
 sing_id: 91563411
 제목: 뿌리 (Feat. JUSTHIS) (Prod. by GroovyRoom)
 가수: Khundi Panda
 index cnt: [2464]
 sing_id: 91563414
 제목: 적외선 카메라
 가수: 원슈타인
 index cnt: [1024]
 sing_id: 91563413
 제목: 부어라 비워라 (Tricker) (Prod. by GroovyRoom)
 가수: 머쉬베놈 (MUSHVENOM)
 index cnt: [1090]
 sing_id: 91529103
 제목: Panorama
 가수: IZ*ONE (아이즈원)
 index cnt: [2938]
 sing_id: 91593370
 제목: 어김없이 이 거리에
 가수: 정승환
 index cnt: [1450]
 sing_id: 91622433
 제목: Hate you
 가수: 백예린 (Yerin Baek)
 index cnt: [1546]
 sing_id: 91620457
 제목: 잘할게
 가수: 이승기
 index cnt: [2500]
 sing_id: 91623159
 제목: 아무도 잠들지 마라 (Nessun D

 sing_id: 92399155
 제목: Body Rhythm
 가수: SHINee (샤이니)
 index cnt: [766]
 sing_id: 92399156
 제목: Attention
 가수: SHINee (샤이니)
 index cnt: [742]
 sing_id: 92399157
 제목: 빈칸 (Kind)
 가수: SHINee (샤이니)
 index cnt: [862]
 sing_id: 92402200
 제목: 꼬리 (TAIL)
 가수: 선미
 index cnt: [1282]
 sing_id: 92419034
 제목: Beautiful Beautiful
 가수: 온앤오프 (ONF)
 index cnt: [1060]
 sing_id: 92413088
 제목: 사랑하는 당신께
 가수: 폴킴
 index cnt: [1330]
 sing_id: 92427194
 제목: 멜로디
 가수: ASH ISLAND
 index cnt: [4397]
 sing_id: 92491599
 제목: 좋은 이별이 있을 리가 없잖아 (Feat. 조현아 of 어반자카파)
 가수: MC몽
 index cnt: [1972]
 sing_id: 92517481
 제목: LOVE DAY (2021) (바른연애 길잡이 X 양요섭, 정은지)
 가수: 양요섭 & 정은지
 index cnt: [4205]
 sing_id: 92522024
 제목: 비로소 너에게 도착했다 (Full Bloom)
 가수: 마크툽 (Maktub)
 index cnt: [994]
 sing_id: 92558379
 제목: 별빛 같은 나의 사랑아
 가수: 임영웅
 index cnt: [4109]
 sing_id: 92561409
 제목: 되풀이
 가수: 임창정
 index cnt: [1786]
 sing_id: 92457797
 제목: 후
 가수: 임영웅
 index cnt: [1882]
 sing_id: 92604203
 제목: On The Ground
 가수: 로제 (ROSE)
 index cnt: [2967, 1071]


 sing_id: 93562243
 제목: 치맛바람 (Chi Mat Ba Ram)
 가수: 브레이브걸스 (Brave girls)
 index cnt: [1709]
 sing_id: 93304399
 제목: 비와 당신
 가수: 이무진
 index cnt: [1691]
 sing_id: 93576068
 제목: Ready to love
 가수: 세븐틴 (SEVENTEEN)
 index cnt: [1282]
 sing_id: 93583239
 제목: 허리춤
 가수: 오반 (OVAN)
 index cnt: [1613]
 sing_id: 93599531
 제목: 하루만 더
 가수: 빅마마 (Big Mama)
 index cnt: [1541]
 sing_id: 93587781
 제목: 가을 우체국 앞에서
 가수: 김대명
 index cnt: [1523]
 sing_id: 93619160
 제목: 바라만 본다
 가수: MSG워너비 (M.O.M)
 index cnt: [1493]
 sing_id: 93619161
 제목: 나를 아는 사람
 가수: MSG워너비 (정상동기)
 index cnt: [1493]
 sing_id: 93633150
 제목: Hello Future
 가수: NCT DREAM
 index cnt: [1018]
 sing_id: 93633532
 제목: 해야 해
 가수: 2PM
 index cnt: [892]
 sing_id: 93569588
 제목: 비 오는 날 듣기 좋은 노래 (Feat. Colde)
 가수: 에픽하이 (EPIK HIGH)
 index cnt: [1421]
 sing_id: 93636377
 제목: 끝사랑
 가수: 임영웅
 index cnt: [1395]
 sing_id: 93578553
 제목: Bk Love
 가수: 임영웅
 index cnt: [1394]
 sing_id: 93640201
 제목: 나는 너 좋아
 가수: 장범준
 index cnt: [1355]
 sing_id: 93664330
 제목: 러브 (Prod. by 로코베

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,...,유통사,발매일_datetime,date_time,date_time_year,date_time_month,date_time_hour,duration,duration_days,duration_seconds,duration_hours
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,...,카카오엔터테인먼트,2019-12-05,2019-12-08 11:00:00,2019,12,11,3 days 11:00:00,3,39600,83
1,2019/12/08/11,2,-,89613039,Blueming,아이유 (IU),0,NaN,NaN,16624334,...,카카오엔터테인먼트,2019-11-18,2019-12-08 11:00:00,2019,12,11,20 days 11:00:00,20,39600,491
2,2019/12/08/11,3,-,89682091,아마두 (Feat. 우원재 & 김효은 & 넉살 & Huckleberry P),염따 & 딥플로우 & 팔로알토 (Paloalto) & The Quiett & 사이먼...,0,NaN,NaN,2971878,...,Dreamus,2019-12-03,2019-12-08 11:00:00,2019,12,11,5 days 11:00:00,5,39600,131
3,2019/12/08/11,4,-,89678583,나의 오랜 연인에게,다비치,0,NaN,NaN,3397457,...,지니뮤직,2019-12-03,2019-12-08 11:00:00,2019,12,11,5 days 11:00:00,5,39600,131
4,2019/12/08/11,6,1,89558702,늦은 밤 너의 집 앞 골목길에서,노을,0,NaN,NaN,20911975,...,카카오엔터테인먼트,2019-11-07,2019-12-08 11:00:00,2019,12,11,31 days 11:00:00,31,39600,755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1858050,2021/10/15/23,217,2,93583239,허리춤,오반 (OVAN),1,1010.0,29.0,6113549,...,카카오엔터테인먼트,2021-06-21,2021-10-15 23:00:00,2021,10,23,116 days 23:00:00,116,82800,2807
1858051,2021/10/15/23,231,9,91290290,가을밤에 든 생각,잔나비,1,974.0,69.0,18878205,...,카카오엔터테인먼트,2020-11-06,2021-10-15 23:00:00,2021,10,23,343 days 23:00:00,343,82800,8255
1858052,2021/10/15/23,233,4,92213348,고백 (바른연애 길잡이 X 허각),허각,1,820.0,26.0,16256986,...,카카오엔터테인먼트,2021-02-04,2021-10-15 23:00:00,2021,10,23,253 days 23:00:00,253,82800,6095
1858053,2021/10/15/23,235,14,94063592,회상,정경호,1,865.0,105.0,2529360,...,"지니뮤직, Stone Music Entertainment",2021-08-20,2021-10-15 23:00:00,2021,10,23,56 days 23:00:00,56,82800,1367


In [12]:
# 설명변수 One-Hot Encoding
music_type_columns, chart_info_df = dummies_columns("앨범 종류", chart_info_df)
music_genre_columns, chart_info_df = dummies_columns("앨범 장르", chart_info_df)
music_style_columns, chart_info_df = dummies_columns("스타일", chart_info_df)

date_time_year_columns, chart_info_df = dummies_columns("date_time_year", chart_info_df)
date_time_month_columns, chart_info_df = dummies_columns("date_time_month", chart_info_df)
date_time_hour_columns, chart_info_df = dummies_columns("date_time_hour", chart_info_df)
chart_info_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,...,date_time_hour_14,date_time_hour_15,date_time_hour_16,date_time_hour_17,date_time_hour_18,date_time_hour_19,date_time_hour_20,date_time_hour_21,date_time_hour_22,date_time_hour_23
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,...,0,0,0,0,0,0,0,0,0,0
1,2019/12/08/11,2,-,89613039,Blueming,아이유 (IU),0,NaN,NaN,16624334,...,0,0,0,0,0,0,0,0,0,0
2,2019/12/08/11,3,-,89682091,아마두 (Feat. 우원재 & 김효은 & 넉살 & Huckleberry P),염따 & 딥플로우 & 팔로알토 (Paloalto) & The Quiett & 사이먼...,0,NaN,NaN,2971878,...,0,0,0,0,0,0,0,0,0,0
3,2019/12/08/11,4,-,89678583,나의 오랜 연인에게,다비치,0,NaN,NaN,3397457,...,0,0,0,0,0,0,0,0,0,0
4,2019/12/08/11,6,1,89558702,늦은 밤 너의 집 앞 골목길에서,노을,0,NaN,NaN,20911975,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1858050,2021/10/15/23,217,2,93583239,허리춤,오반 (OVAN),1,1010.0,29.0,6113549,...,0,0,0,0,0,0,0,0,0,1
1858051,2021/10/15/23,231,9,91290290,가을밤에 든 생각,잔나비,1,974.0,69.0,18878205,...,0,0,0,0,0,0,0,0,0,1
1858052,2021/10/15/23,233,4,92213348,고백 (바른연애 길잡이 X 허각),허각,1,820.0,26.0,16256986,...,0,0,0,0,0,0,0,0,0,1
1858053,2021/10/15/23,235,14,94063592,회상,정경호,1,865.0,105.0,2529360,...,0,0,0,0,0,0,0,0,0,1


In [13]:
music_genre_columns

['앨범 장르_OST',
 '앨범 장르_댄스/팝',
 '앨범 장르_락/메탈',
 '앨범 장르_랩/힙합',
 '앨범 장르_발라드',
 '앨범 장르_성인가요',
 '앨범 장르_알앤비/소울',
 '앨범 장르_인디',
 '앨범 장르_일렉트로닉',
 '앨범 장르_팝',
 '앨범 장르_포크/어쿠스틱']

## 학습 데이터셋 생성

In [17]:
data_num = 30
meta_columns = music_type_columns + music_genre_columns + music_style_columns + \
               date_time_year_columns + date_time_month_columns + date_time_hour_columns + ["duration_hours"]

x_train_list = []
y_train_list = []

x_test_list = []
y_test_list = []

for sing_id, compare_idx_list in sing_id_dict.items():
    x_data_list = []
    y_data_list = []
    chart_one_df = chart_info_df[chart_info_df["sing_id"] == sing_id].reset_index(drop=True)

    for c_idx, compare_index in enumerate(compare_idx_list):
        chart_final_df = chart_one_df.iloc[compare_index[0]:compare_index[-1]].reset_index(drop=True)

        for i in range(0, chart_final_df.shape[0] - data_num):
            null_check_df = chart_final_df.iloc[i:i + data_num + 1][meta_columns + ["play_in_term"]]
            if null_check_df.isnull().values.any() or chart_final_df.iloc[i + data_num]["play_in_term"] == 0:
                continue

            x_data_meta = chart_final_df.iloc[i][meta_columns].values
            x_data_play = chart_final_df.iloc[i:i + data_num]["play_in_term"].values

            x_data_tmp = np.concatenate((x_data_meta, x_data_play))
            x_data_dims = np.expand_dims(x_data_tmp, axis=0)

            y_data_tmp = chart_final_df.iloc[i + data_num]["play_in_term"]
            y_data_dims = np.expand_dims(y_data_tmp, axis=0)

            x_data_list.append(x_data_dims)
            y_data_list.append(y_data_dims)

    data_cnt = int(len(x_data_list) * 0.8)
    x_train_list = x_train_list + x_data_list[:data_cnt]
    y_train_list = y_train_list + y_data_list[:data_cnt]

    x_test_list = x_test_list + x_data_list[data_cnt:]
    y_test_list = y_test_list + y_data_list[data_cnt:]

    print("=" * 50)
    print(" sing_id: {}\n 제목: {}\n 가수: {}\n Dataset count: {}".format(
        sing_id, chart_one_df.iloc[0]["subject"], chart_one_df.iloc[0]["singer"], len(x_data_list)))

 sing_id: 89688529
 제목: 십이월 이십오일의 고백
 가수: 정승환
 Dataset count: 1785
 sing_id: 89613039
 제목: Blueming
 가수: 아이유 (IU)
 Dataset count: 6754
 sing_id: 89682091
 제목: 아마두 (Feat. 우원재 & 김효은 & 넉살 & Huckleberry P)
 가수: 염따 & 딥플로우 & 팔로알토 (Paloalto) & The Quiett & 사이먼 도미닉
 Dataset count: 3673
 sing_id: 89678583
 제목: 나의 오랜 연인에게
 가수: 다비치
 Dataset count: 4678
 sing_id: 89558702
 제목: 늦은 밤 너의 집 앞 골목길에서
 가수: 노을
 Dataset count: 6754
 sing_id: 89598605
 제목: HIP
 가수: 마마무(Mamamoo)
 Dataset count: 6160
 sing_id: 89671315
 제목: 헤어진 우리가 지켜야 할 것들
 가수: 김나영 & 양다일
 Dataset count: 3338
 sing_id: 89530447
 제목: Love poem
 가수: 아이유 (IU)
 Dataset count: 6754
 sing_id: 89382909
 제목: 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지
 가수: AKMU (악뮤)
 Dataset count: 4678
 sing_id: 89320910
 제목: 흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야
 가수: 장범준
 Dataset count: 4678
 sing_id: 89602798
 제목: Show Yourself
 가수: Idina Menzel & Evan Rachel Wood
 Dataset count: 1430
 sing_id: 89572801
 제목: 이 번호로 전화해줘
 가수: 바이브
 Dataset count: 2731
 sing_id: 89380047
 제목: 조금 취했어 (Prod. by 2soo)

 sing_id: 89374289
 제목: Memories
 가수: Maroon 5
 Dataset count: 4300
 sing_id: 89378490
 제목: 나만 아픈 일
 가수: 이해리 (다비치)
 Dataset count: 1530
 sing_id: 89369155
 제목: 가끔 이러다
 가수: 펀치 (Punch)
 Dataset count: 697
 sing_id: 89391731
 제목: 천둥벌거숭이 (Feat. Jvcki Wai & 염따)
 가수: 지코 (ZICO)
 Dataset count: 697
 sing_id: 88824340
 제목: FANCY
 가수: TWICE (트와이스)
 Dataset count: 1785
 sing_id: 88715422
 제목: 안녕이라는 말
 가수: 송이한
 Dataset count: 697
 sing_id: 89258787
 제목: ICY
 가수: ITZY (있지)
 Dataset count: 697
 sing_id: 89408689
 제목: 이별행동
 가수: 이우
 Dataset count: 1785
 sing_id: 89623205
 제목: 우리 왜 헤어져야 해
 가수: 신예영
 Dataset count: 6375
 sing_id: 89226517
 제목: 내 안부
 가수: 이우
 Dataset count: 1508
 sing_id: 88773174
 제목: 소우주 (Mikrokosmos)
 가수: 방탄소년단
 Dataset count: 697
 sing_id: 89671630
 제목: 빌었어
 가수: 창모 (CHANGMO)
 Dataset count: 4678
 sing_id: 88728543
 제목: 사월이 지나면 우리 헤어져요 (Beautiful goodbye)
 가수: 첸 (CHEN)
 Dataset count: 697
 sing_id: 89276181
 제목: 그 끝에 그대
 가수: 청하
 Dataset count: 697
 sing_id: 88607738
 제목: 달라달라
 가수: ITZY 

 sing_id: 89830405
 제목: 돌덩이
 가수: 하현우 (국카스텐)
 Dataset count: 5271
 sing_id: 89831932
 제목: 둘만의 세상으로 가
 가수: Crush
 Dataset count: 2606
 sing_id: 89831917
 제목: 잘 지내고 있는지 궁금해
 가수: V.O.S
 Dataset count: 5264
 sing_id: 89859975
 제목: 마음을 드려요
 가수: 아이유 (IU)
 Dataset count: 5879
 sing_id: 89858874
 제목: 그때 그 아인
 가수: 김필
 Dataset count: 5879
 sing_id: 89863137
 제목: FIESTA
 가수: IZ*ONE (아이즈원)
 Dataset count: 4899
 sing_id: 89786517
 제목: Black Swan
 가수: 방탄소년단
 Dataset count: 5213
 sing_id: 89894708
 제목: 어느 60대 노부부이야기
 가수: 임영웅
 Dataset count: 4776
 sing_id: 89883239
 제목: Filter
 가수: 방탄소년단
 Dataset count: 4767
 sing_id: 89883240
 제목: 시차
 가수: 방탄소년단
 Dataset count: 3804
 sing_id: 89883243
 제목: 욱 (UGH!)
 가수: 방탄소년단
 Dataset count: 2786
 sing_id: 89883241
 제목: Louder than bombs
 가수: 방탄소년단
 Dataset count: 2570
 sing_id: 89883246
 제목: 친구
 가수: 방탄소년단
 Dataset count: 5102
 sing_id: 89883237
 제목: Interlude : Shadow
 가수: 방탄소년단
 Dataset count: 2450
 sing_id: 89883245
 제목: Inner Child
 가수: 방탄소년단
 Dataset count: 2906
 

 sing_id: 90173969
 제목: Happy
 가수: 태연 (TAEYEON)
 Dataset count: 2728
 sing_id: 90173917
 제목: 은하수
 가수: 길구봉구
 Dataset count: 843
 sing_id: 90146199
 제목: 서면역에서
 가수: 순순희
 Dataset count: 4799
 sing_id: 90178428
 제목: 에잇 (Prod. & Feat. SUGA of BTS)
 가수: 아이유 (IU)
 Dataset count: 3972
 sing_id: 88323996
 제목: Comethru
 가수: Jeremy Zucker
 Dataset count: 6372
 sing_id: 90194896
 제목: Zombie
 가수: DAY6 (데이식스)
 Dataset count: 1776
 sing_id: 90198070
 제목: I'm in Trouble
 가수: 뉴이스트
 Dataset count: 753
 sing_id: 90182881
 제목: 나비와 고양이 (Feat. 백현 (BAEKHYUN))
 가수: 볼빨간사춘기
 Dataset count: 3230
 sing_id: 90208247
 제목: 품
 가수: 볼빨간사춘기
 Dataset count: 2704
 sing_id: 90210693
 제목: 밤이 깊었네 (Drama Ver.)
 가수: 미도와 파라솔
 Dataset count: 1353
 sing_id: 90235529
 제목: 자나깨나 (Feat. 조이 of Red Velvet)
 가수: Crush
 Dataset count: 1560
 sing_id: 90250302
 제목: 사랑하게 될 줄 알았어
 가수: 전미도
 Dataset count: 7468
 sing_id: 90253075
 제목: 취했나봐
 가수: 황인욱
 Dataset count: 3808
 sing_id: 90257120
 제목: Candy
 가수: 백현 (BAEKHYUN)
 Dataset count: 2694
 sing_

 sing_id: 90612173
 제목: 깨워 (Who U Are)
 가수: 강다니엘
 Dataset count: 1446
 sing_id: 90641203
 제목: 아틀란티스 소녀 (Atlantis Princess)
 가수: 볼빨간사춘기
 Dataset count: 1775
 sing_id: 90641114
 제목: 나가
 가수: 먼데이 키즈 (Monday Kiz) & 이예준
 Dataset count: 753
 sing_id: 90661587
 제목: When We Disco (Duet With 선미)
 가수: 박진영
 Dataset count: 5409
 sing_id: 90677250
 제목: 오래된 노래
 가수: 임영웅
 Dataset count: 4161
 sing_id: 90676965
 제목: Not Shy
 가수: ITZY (있지)
 Dataset count: 2343
 sing_id: 90705547
 제목: Dynamite
 가수: 방탄소년단
 Dataset count: 1444
 sing_id: 90709386
 제목: 안녕 못 해
 가수: 김재환
 Dataset count: 1359
 sing_id: 90709438
 제목: 내 마음이 움찔했던 순간 (취향저격 그녀 X 규현)
 가수: 규현 (KYUHYUN)
 Dataset count: 7312
 sing_id: 90743704
 제목: 듣고 싶던 말
 가수: 허각
 Dataset count: 1294
 sing_id: 90751091
 제목: Ice Cream (With Selena Gomez)
 가수: BLACKPINK
 Dataset count: 1276
 sing_id: 90751198
 제목: 그 중에 그대를 만나
 가수: 임영웅
 Dataset count: 1220
 sing_id: 90751194
 제목: 그런 사람 또 없습니다
 가수: 임영웅
 Dataset count: 801
 sing_id: 90788463
 제목: 만개 (Prod. 신지후)
 가수: 김호중
 Data

 sing_id: 91423134
 제목: Freak (Prod. by Slom)
 가수: 릴보이 (lIlBOI) & 원슈타인 & Chillin Homie & 스카이민혁 (Skyminhyuk)
 Dataset count: 2337
 sing_id: 91423131
 제목: 원해 (Feat. 팔로알토) (Prod. by 코드 쿤스트)
 가수: 스윙스 (Swings) & 맥대디 (Mckdaddy) & Khakii (카키) & 래원 (Layone)
 Dataset count: 1497
 sing_id: 91423132
 제목: 윈윈 (Feat. 개코 & BewhY) (Prod. by BewhY)
 가수: 허성현 (Rose de Penny) & dsel (디젤) & 가오가이 (kaogaii) & Untell (언텔)
 Dataset count: 777
 sing_id: 91466463
 제목: 혼술하고 싶은 밤
 가수: 벤
 Dataset count: 4251
 sing_id: 91552332
 제목: 한 번에 알아본 사랑 (바른연애 길잡이 X 양요섭)
 가수: 양요섭
 Dataset count: 1123
 sing_id: 91551008
 제목: 그날에 나는 맘이 편했을까
 가수: 이예준
 Dataset count: 6348
 sing_id: 91563412
 제목: 내일이 오면 (Feat. 기리보이 & BIG Naughty (서동현))
 가수: 릴보이 (lIlBOI)
 Dataset count: 3321
 sing_id: 91563409
 제목: Achoo (Feat. pH-1 & HAON) (Prod. by GroovyRoom)
 가수: 미란이 (Mirani)
 Dataset count: 2361
 sing_id: 91563411
 제목: 뿌리 (Feat. JUSTHIS) (Prod. by GroovyRoom)
 가수: Khundi Panda
 Dataset count: 2433
 sing_id: 91563414
 제목: 적외선 카메라
 가수: 원슈타인
 Dat

 sing_id: 92399149
 제목: Don't Call Me
 가수: SHINee (샤이니)
 Dataset count: 2931
 sing_id: 92399150
 제목: Heart Attack
 가수: SHINee (샤이니)
 Dataset count: 903
 sing_id: 92399151
 제목: Marry You
 가수: SHINee (샤이니)
 Dataset count: 781
 sing_id: 92399152
 제목: CØDE
 가수: SHINee (샤이니)
 Dataset count: 780
 sing_id: 92399153
 제목: I Really Want You
 가수: SHINee (샤이니)
 Dataset count: 734
 sing_id: 92399154
 제목: Kiss Kiss
 가수: SHINee (샤이니)
 Dataset count: 735
 sing_id: 92399155
 제목: Body Rhythm
 가수: SHINee (샤이니)
 Dataset count: 735
 sing_id: 92399156
 제목: Attention
 가수: SHINee (샤이니)
 Dataset count: 711
 sing_id: 92399157
 제목: 빈칸 (Kind)
 가수: SHINee (샤이니)
 Dataset count: 831
 sing_id: 92402200
 제목: 꼬리 (TAIL)
 가수: 선미
 Dataset count: 1251
 sing_id: 92419034
 제목: Beautiful Beautiful
 가수: 온앤오프 (ONF)
 Dataset count: 1029
 sing_id: 92413088
 제목: 사랑하는 당신께
 가수: 폴킴
 Dataset count: 1299
 sing_id: 92427194
 제목: 멜로디
 가수: ASH ISLAND
 Dataset count: 4366
 sing_id: 92491599
 제목: 좋은 이별이 있을 리가 없잖아 (Feat. 조현아 of 어반자카파)
 가수: M

 sing_id: 93049952
 제목: 바보같지만
 가수: 임영웅
 Dataset count: 1011
 sing_id: 93409560
 제목: 벌써일년
 가수: 반하나 & MJ (써니사이드)
 Dataset count: 1612
 sing_id: 93409581
 제목: 안녕 (Hello)
 가수: 조이 (JOY)
 Dataset count: 2086
 sing_id: 93495902
 제목: 내 입술 따뜻한 커피처럼
 가수: 청하 & Colde (콜드)
 Dataset count: 820
 sing_id: 93506686
 제목: Alcohol-Free
 가수: TWICE (트와이스)
 Dataset count: 1842
 sing_id: 93522004
 제목: 추적이는 여름 비가 되어
 가수: 장범준
 Dataset count: 1822
 sing_id: 93524409
 제목: 오늘따라 더 미운 그대가
 가수: 전상근
 Dataset count: 1276
 sing_id: 93562243
 제목: 치맛바람 (Chi Mat Ba Ram)
 가수: 브레이브걸스 (Brave girls)
 Dataset count: 1678
 sing_id: 93304399
 제목: 비와 당신
 가수: 이무진
 Dataset count: 1660
 sing_id: 93576068
 제목: Ready to love
 가수: 세븐틴 (SEVENTEEN)
 Dataset count: 1251
 sing_id: 93583239
 제목: 허리춤
 가수: 오반 (OVAN)
 Dataset count: 1582
 sing_id: 93599531
 제목: 하루만 더
 가수: 빅마마 (Big Mama)
 Dataset count: 1510
 sing_id: 93587781
 제목: 가을 우체국 앞에서
 가수: 김대명
 Dataset count: 1492
 sing_id: 93619160
 제목: 바라만 본다
 가수: MSG워너비 (M.O.M)
 Dataset count: 1462
 s

In [18]:
x_train = np.concatenate(x_train_list)
y_train = np.concatenate(y_train_list)

x_test = np.concatenate(x_test_list)
y_test = np.concatenate(y_test_list)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1003363, 107), (1003363,), (251087, 107), (251087,))

## 학습 데이터셋 저장

In [20]:
with h5py.File('../dataset/train_test_data.hdf5','w') as f2:
    f2.create_dataset('x_train', data=x_train.astype(np.float64))
    f2.create_dataset('y_train', data=y_train)
    
    f2.create_dataset('x_test', data=x_test.astype(np.float64))
    f2.create_dataset('y_test', data=y_test)